In [3]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/encoded.csv", nrows=10000)
#df.loc[400:,"enc_feature_0":"enc_feature_49"]
#strip = 1 + df.shape[0] % 200
#df.loc[:df.shape[0]-strip]
#df["enc_feature_0", "feature_0"]
df.insert(3, "trade", None)
df.loc[df["resp"] <= 0, "trade"] = -1
df.loc[df["resp"] > 0, "trade"] = 1
#df = df[df["weight"] > 0]
df["weight"].min()

0.0

In [8]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-0.4211, -1.3836,  0.0389])

In [1]:
VALIDATION_SIZE = 1000
p=0.1
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000, skiprows=lambda i: i > 0 and np.random.random() > p)


NameError: name 'pd' is not defined

In [2]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

NameError: name 'df' is not defined

In [96]:
feature_0_pos = df.columns.get_loc("feature_0")
df.insert(feature_0_pos, "w_date", None)
df.insert(feature_0_pos, "w_ts_id", None)
df.loc[:, "w_date"] = (df["date"] / df["date"].max()) * 10
for one in range(df["date"].max()+1):
    min_id = df.loc[df["date"] == one, :]["ts_id"].min()
    max_id = df.loc[df["date"] == one, :]["ts_id"].max()
    distance = max_id - min_id
    df.loc[df["date"] == one, "w_ts_id"] = (df.loc[df["date"] == one, :]["ts_id"] - min_id) / distance

df.iloc[4000]["w_ts_id"]

0.6587915954559164

In [7]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [8]:
df.iloc[2]["resp"]

-0.009634158690232266

In [9]:
df.iloc[2]["feature_0":"feature_129"].values

array([ 1.00000000e+00, -1.13518959e+00, -1.61827057e+00,  4.54450598e-01,
        2.80194598e-01,  8.34325776e-01,  6.00991096e-01, -7.52922825e-01,
       -7.60245730e-01, -1.26065357e+00, -1.16282365e+00,  3.28691328e-01,
        1.57604453e-01, -2.00281266e+00, -1.54143486e+00, -8.06410352e-01,
       -1.44557732e+00,  8.54325656e-01,  8.53438605e-01,  2.70115762e-01,
        1.22521804e-01,  1.07888779e+00,  4.84688120e-01,  2.30981202e-01,
        9.86927946e-02,  2.75536780e-01,  1.03283244e-01, -5.49189303e-01,
       -7.48393220e-01, -5.29760682e-01, -4.92395888e-01, -6.75317837e-01,
       -1.07180458e+00, -1.06146682e+00, -9.24106051e-01, -7.42370466e-01,
       -4.48241035e-01,  1.07317754e+00,  9.00970932e-01, -2.40498460e-01,
       -3.47299546e-01,  2.17191018e+00, -3.95957967e-01,  1.14263502e+00,
        4.47306184e-01,  5.67436716e-01,  5.70503068e-01,  7.82952089e-01,
        4.61510203e-01,  1.99919843e-01,  1.22670170e+00,  2.65245924e+00,
        1.34119908e+00, -

In [10]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


<ipython-input-10-ae5219358239>:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [11]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [12]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [13]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [14]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

In [ ]:
accuracy_list

In [ ]:
loss_list